<a href="https://colab.research.google.com/github/kylo-dev/gachon-machinelearning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

앙상블 (Ensemble):

앙상블은 여러 개의 기본 모델을 조합하여 더 강력하고 일반화된 예측 모델을 만드는 기법

앙상블은 "다수결 투표"와 같은 방식으로 여러 모델의 예측을 결합하여 더 정확한 예측을 수행합니다.

앙상블은 다양한 모델의 다양성을 활용하여 과적합을 줄이고 예측 성능을 향상시킵니다.

## 랜덤포레스트

* 랜덤 포레스트는 앙상블(Ensemble) 학습 기법의 하나로, 여러 개의 결정 트리 모델을 조합하여 더 강력하고 안정적인 예측 모델을 만드는 알고리즘입니다.

* 랜덤 포레스트는 결정 트리의 과적합 문제를 완화하고 모델의 일반화 성능을 향상시킵니다.
* 주요 알고리즘
    * 무작위로 데이터를 샘플링하여 부트스트랩(Bootstrap) 데이터셋을 생성합니다.
    * 각 결정 트리는 부트스트랩 데이터셋을 사용하여 훈련됩니다.
    * 노드를 분할할 때, 무작위로 선택한 특성(feature) 하위 집합을 고려하여 분할합니다.
    * 모든 결정 트리의 예측을 조합하여 최종 예측을 수행합니다.

---

### 부트스트랩 (Bootstrap):

* 부트스트랩은 통계학과 머신러닝에서 사용되는 리샘플링 기법 중 하나로, 주어진 데이터셋으로부터 무작위로 복원추출을 통해 샘플을 생성하는 과정을 나타냅니다.

* 부트스트랩 샘플링은 주어진 데이터셋에서 중복을 허용하며 랜덤하게 데이터를 선택합니다.

* 부트스트랩을 사용하면 데이터셋으로부터 여러 개의 부분집합을 생성하여 통계적 추정이나 모델 평가를 수행할 수 있습니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


### RandomForestClassifier의 두 가지 파라미터인 oob_score와 n_jobs

1. oob_score (Out-of-Bag Score):

    랜덤 포레스트는 부트스트랩(Bootstrap) 샘플링을 사용하여 각 결정 트리(Decision Tree)를 훈련합니다. 이로 인해 일부 데이터 포인트는 각 결정 트리의 훈련에 사용되지 않게 됩니다. 이러한 데이터 포인트를 "Out-of-Bag" 데이터라고 합니다.

    oob_score=True로 설정하면 랜덤 포레스트는 각 결정 트리의 훈련에 사용되지 않은 Out-of-Bag 데이터를 사용하여 모델의 성능을 평가합니다. 이를 통해 교차 검증 없이 모델의 성능을 추정할 수 있습니다.

2. n_jobs (Number of Jobs):

    n_jobs는 랜덤 포레스트 모델을 훈련할 때 병렬 처리를 수행할 때 사용되는 파라미터입니다.

    일반적으로, n_jobs=-1로 설정하면 가능한 모든 CPU 코어를 활용하여 병렬 처리를 수행하게 됩니다. 이렇게 설정하면 모델 훈련이 빨라질 수 있습니다.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

랜덤 분할(Random Splitting):

엑스트라 트리는 랜덤 포레스트와 마찬가지로 결정 트리(Decision Tree)의 앙상블을 사용합니다.

하지만 랜덤 포레스트와 달리, 엑스트라 트리는 노드를 분할할 때 무작위로 분할 기준(threshold)을 선택합니다.

이러한 무작위 분할로 인해 결정 트리의 다양성이 증가하며 과적합을 줄일 수 있습니다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.11949946 0.74871836 0.13178218]


## 히스토그램 기반 부스팅

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

## XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


## LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
